In [1]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [27]:
data = pd.read_csv("jester-data-1.csv",header=None)
d = data.to_latex()

In [28]:
text_file = open("Output.txt", "w")
text_file.write(d)
text_file.close()

In [29]:
training = data[:int(0.8*len(data))]

In [30]:
test=data[int(0.8*len(data)+1):int(0.9*len(data)+1)]

In [31]:
validation =data[int(0.9*len(data)+1):]

In [32]:
n_features = 2

user_ratings = training.values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_features))
latent_item_features = np.random.random((user_ratings.shape[1],n_features))

In [36]:
def predict_rating(user_id,item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)

def train(user_id, item_id, rating,alpha = 0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  ( prediction_rating- rating );
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    


def sgd(iterations = 30):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(0,iterations):
        error = []
        for user_id in range(0,latent_user_preferences.shape[0]):
            for item_id in range(0,latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if(rating != 99):
                    err = train(user_id,item_id,rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        #if(iteration%10000 == 0 ):
        print mse
                    
                    

In [37]:
#train(0,0,10)
sgd()
#user_preferences 
#item_features

25.7200755254
24.9786428141
24.6751196533
24.4318269015
24.1961458518
23.959915306
23.7226181073
23.4854904046
23.2502572942
23.0187479675
22.7927280604
22.5738042356
22.3633638061
22.1625384571
21.972187652
21.7928984992
21.6249987006
21.4685789548
21.3235211992
21.1895293681
21.0661598476
20.9528493875
20.8489388049
20.7536913039
20.6663046132
20.5859164027
20.5116025986
20.4423683236
20.3771313162
20.3146979307


In [38]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[-0.099, -0.12 , -0.278, ..., -0.047, -0.128, -0.032],
       [ 1.495,  1.42 ,  1.301, ...,  1.759,  1.215,  2.09 ],
       [ 4.29 ,  4.068,  3.682, ...,  5.063,  3.472,  6.025],
       ..., 
       [ 3.519,  3.379,  3.325, ...,  4.044,  2.924,  4.773],
       [ 1.67 ,  1.588,  1.467, ...,  1.959,  1.36 ,  2.327],
       [ 2.073,  2.037,  2.293, ...,  2.261,  1.805,  2.626]])

In [39]:
values = [zip(user_ratings[i], predictions[i]) for i in range(0,predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = data.columns
comparison_data.applymap(lambda (x,y): "(%2.3f|%2.3f)"%(x,y))

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,(-7.820|-0.099),(8.790|-0.120),(-9.660|-0.278),(-8.160|-0.224),(-7.520|-0.088),(-8.500|0.014),(-9.850|-0.117),(4.170|-0.095),(-8.980|-0.269),(-4.760|-0.059),...,(2.820|-0.119),(99.000|-0.144),(99.000|-0.140),(99.000|-0.138),(99.000|-0.032),(99.000|-0.060),(-5.630|-0.037),(99.000|-0.047),(99.000|-0.128),(99.000|-0.032)
1,(4.080|1.495),(-0.290|1.420),(6.360|1.301),(4.370|0.583),(-2.380|1.208),(-9.660|2.377),(-0.730|0.579),(-5.340|0.173),(8.880|0.909),(9.220|1.721),...,(2.820|2.077),(-4.950|1.853),(-0.290|2.399),(7.860|1.648),(-0.190|1.625),(-2.140|1.953),(3.060|1.929),(0.340|1.759),(-4.320|1.215),(1.070|2.090)
2,(99.000|4.290),(99.000|4.068),(99.000|3.682),(99.000|1.625),(9.030|3.464),(9.270|6.863),(9.030|1.642),(9.270|0.475),(99.000|2.552),(99.000|4.951),...,(99.000|5.963),(99.000|5.311),(99.000|6.887),(9.080|4.722),(99.000|4.682),(99.000|5.620),(99.000|5.559),(99.000|5.063),(99.000|3.472),(99.000|6.025)
3,(99.000|2.512),(8.350|2.414),(99.000|2.384),(99.000|1.171),(1.800|2.038),(8.160|3.816),(-2.820|1.055),(6.210|0.378),(99.000|1.742),(1.840|2.835),...,(99.000|3.469),(99.000|3.135),(99.000|4.010),(0.530|2.799),(99.000|2.652),(99.000|3.209),(99.000|3.148),(99.000|2.883),(99.000|2.090),(99.000|3.402)
4,(8.500|3.042),(4.610|2.862),(-4.170|2.451),(-5.390|0.998),(1.360|2.449),(1.600|5.007),(7.040|1.099),(4.610|0.267),(-0.440|1.636),(5.730|3.555),...,(5.190|4.243),(5.580|3.747),(4.270|4.898),(5.190|3.324),(5.730|3.382),(1.550|4.041),(3.110|4.016),(6.550|3.647),(1.800|2.422),(1.600|4.360)
5,(-6.170|-2.596),(-3.540|-2.466),(0.440|-2.260),(-8.500|-1.014),(-7.090|-2.098),(-4.320|-4.124),(-8.690|-1.007),(-0.870|-0.301),(-6.650|-1.580),(-1.800|-2.987),...,(-3.540|-3.605),(-6.890|-3.217),(-0.680|-4.164),(-2.960|-2.862),(-2.180|-2.820),(-3.350|-3.389),(0.050|-3.348),(-9.080|-3.052),(-5.050|-2.109),(-3.450|-3.628)
6,(99.000|2.839),(99.000|2.663),(99.000|2.225),(99.000|0.871),(8.590|2.283),(-9.850|4.729),(7.720|1.000),(8.790|0.221),(99.000|1.459),(99.000|3.336),...,(99.000|3.966),(99.000|3.491),(99.000|4.578),(99.000|3.093),(99.000|3.181),(2.330|3.795),(99.000|3.778),(99.000|3.427),(99.000|2.245),(99.000|4.104)
7,(6.840|-0.594),(3.160|-0.535),(9.170|-0.307),(-6.210|-0.029),(-8.160|-0.471),(-1.700|-1.131),(9.270|-0.144),(1.410|0.023),(-5.190|-0.133),(-4.420|-0.743),...,(7.230|-0.845),(-1.120|-0.713),(-0.100|-0.974),(-5.680|-0.624),(-3.160|-0.728),(-3.350|-0.851),(2.140|-0.866),(-0.050|-0.774),(1.310|-0.430),(0.000|-0.947)
8,(-3.790|0.661),(-3.540|0.613),(-9.420|0.471),(-6.890|0.157),(-8.740|0.530),(-0.290|1.143),(-5.290|0.214),(-8.930|0.031),(-7.860|0.288),(-1.600|0.790),...,(4.370|0.928),(-0.290|0.807),(4.170|1.070),(-0.290|0.713),(-0.290|0.759),(-0.290|0.900),(-0.290|0.902),(-0.290|0.815),(-3.400|0.511),(-4.950|0.982)
9,(3.010|3.015),(5.150|2.850),(5.150|2.525),(3.010|1.081),(6.410|2.432),(5.150|4.878),(8.930|1.128),(2.520|0.306),(3.010|1.725),(8.160|3.497),...,(99.000|4.196),(4.470|3.725),(99.000|4.845),(99.000|3.309),(99.000|3.315),(99.000|3.971),(99.000|3.936),(99.000|3.581),(99.000|2.424),(99.000|4.269)


In [40]:
comparison_data

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,"(-7.82, -0.0987448470908)","(8.79, -0.120359423666)","(-9.66, -0.277806409195)","(-8.16, -0.224129277697)","(-7.52, -0.0880659721997)","(-8.5, 0.0135772351705)","(-9.85, -0.116783516562)","(4.17, -0.0954677158827)","(-8.98, -0.269027478408)","(-4.76, -0.0594469081706)",...,"(2.82, -0.118673751189)","(99.0, -0.143644215749)","(99.0, -0.139666228294)","(99.0, -0.137539224378)","(99.0, -0.0317870990606)","(99.0, -0.0602228191107)","(-5.63, -0.0369711266703)","(99.0, -0.0470426286438)","(99.0, -0.128019083497)","(99.0, -0.0315881319291)"
1,"(4.08, 1.49548664833)","(-0.29, 1.42042701528)","(6.36, 1.30093826412)","(4.37, 0.583245133428)","(-2.38, 1.2082735592)","(-9.66, 2.37653592904)","(-0.73, 0.579460526568)","(-5.34, 0.173093338926)","(8.88, 0.908692788666)","(9.22, 1.72075330968)",...,"(2.82, 2.07666838241)","(-4.95, 1.85305078398)","(-0.29, 2.39870948071)","(7.86, 1.64842281304)","(-0.19, 1.62497579275)","(-2.14, 1.9525108044)","(3.06, 1.92925197117)","(0.34, 1.75860660695)","(-4.32, 1.21456083431)","(1.07, 2.09038069335)"
2,"(99.0, 4.29047830427)","(99.0, 4.0682046757)","(99.0, 3.68226768791)","(99.0, 1.62488158811)","(9.03, 3.46431662891)","(9.27, 6.86264132736)","(9.03, 1.64196067589)","(9.27, 0.474674110062)","(99.0, 2.55246850309)","(99.0, 4.95088863305)",...,"(99.0, 5.96267210162)","(99.0, 5.31075871089)","(99.0, 6.88666312313)","(9.08, 4.72175903412)","(99.0, 4.68167323306)","(99.0, 5.619579014)","(99.0, 5.55851535865)","(99.0, 5.06337565933)","(99.0, 3.47204067198)","(99.0, 6.02496531949)"
3,"(99.0, 2.51220780761)","(8.35, 2.41355737291)","(99.0, 2.38357435054)","(99.0, 1.1714561014)","(1.8, 2.03828633828)","(8.16, 3.81617718066)","(-2.82, 1.05450178462)","(6.21, 0.377558264987)","(99.0, 1.74233709544)","(1.84, 2.83468140857)",...,"(99.0, 3.46946293862)","(99.0, 3.13485126466)","(99.0, 4.01016650376)","(0.53, 2.79875337982)","(99.0, 2.65175541667)","(99.0, 3.20899992623)","(99.0, 3.14750253492)","(99.0, 2.882878776)","(99.0, 2.08967850029)","(99.0, 3.40163035008)"
4,"(8.5, 3.04175318678)","(4.61, 2.86209378176)","(-4.17, 2.45110389088)","(-5.39, 0.997736528038)","(1.36, 2.44915795392)","(1.6, 5.0069648182)","(7.04, 1.09883075459)","(4.61, 0.266694605357)","(-0.44, 1.63590246178)","(5.73, 3.55496543286)",...,"(5.19, 4.24258875022)","(5.58, 3.74739561315)","(4.27, 4.89788172567)","(5.19, 3.32366784583)","(5.73, 3.38183188361)","(1.55, 4.04110661244)","(3.11, 4.01585904306)","(6.55, 3.64709706084)","(1.8, 2.42178185616)","(1.6, 4.35986351853)"
5,"(-6.17, -2.59613626284)","(-3.54, -2.46611132597)","(0.44, -2.26040572639)","(-8.5, -1.01443855907)","(-7.09, -2.09762624804)","(-4.32, -4.12384257723)","(-8.69, -1.00675146017)","(-0.87, -0.301363319302)","(-6.65, -1.57965365152)","(-1.8, -2.98662982906)",...,"(-3.54, -3.60486425159)","(-6.89, -3.21708284459)","(-0.68, -4.16391861706)","(-2.96, -2.86192984823)","(-2.18, -2.82013934006)","(-3.35, -3.38880488922)","(0.05, -3.34820148461)","(-9.08, -3.05218697323)","(-5.05, -2.10895324952)","(-3.45, -3.62775074633)"
6,"(99.0, 2.83923502149)","(99.0, 2.66285171858)","(99.0, 2.22517942365)","(99.0, 0.870632150679)","(8.59, 2.28338607464)","(-9.85, 4.72933654525)","(7.72, 1.00000348561)","(8.79, 0.221467373759)","(99.0, 1.4586572269)","(99.0, 3.33598583386)",...,"(99.0, 3.96614924754)","(99.0, 3.49093685427)","(99.0, 4.57790192447)","(99.0, 3.09299963644)","(99.0, 3.18135372896)","(2.33, 3.79450855979)","(99.0, 3.7780405493)","(99.0, 3.42685516237)","(99.0, 2.24490833967)","(99.0, 4.10437876888)"
7,"(6.84, -0.594334733266)","(3.16, -0.535452740347)","(9.17, -0.307189556589)","(-6.21, -0.0288408523843)","(-8.16, -0.471131708895)","(-1.7, -1.1308981896)","(9.27, -0.14443345059)","(1.41, 0.0230957126666)","(-5.19, -0.132584553208)","(-4.42, -0.743090623895)",...,"(7.23, -0.845475555305)","(-1.12, -0.713159486236)","(-0.1, -0.973757574208)","(-5.68, -0.623738538022)","(-3.16, -0.728354951273)","(-3.35, -0.851078679155)","(2.14, -0.8

In [41]:
predictions

array([[-0.099, -0.12 , -0.278, ..., -0.047, -0.128, -0.032],
       [ 1.495,  1.42 ,  1.301, ...,  1.759,  1.215,  2.09 ],
       [ 4.29 ,  4.068,  3.682, ...,  5.063,  3.472,  6.025],
       ..., 
       [ 3.519,  3.379,  3.325, ...,  4.044,  2.924,  4.773],
       [ 1.67 ,  1.588,  1.467, ...,  1.959,  1.36 ,  2.327],
       [ 2.073,  2.037,  2.293, ...,  2.261,  1.805,  2.626]])

In [42]:
predictions.index(max(predictions))
predictions.index(min(predictions))

AttributeError: 'numpy.ndarray' object has no attribute 'index'

-17.965201897139853